<a href="https://colab.research.google.com/github/Kavyapm1960/project/blob/main/trans_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import corpus_bleu

# Load and preprocess the data
with open('/content/drive/MyDrive/project-trans/small_vocab_en.txt', 'r', encoding='utf-8') as file:
    english_sentences = file.read().split('\n')

with open('/content/drive/MyDrive/project-trans/small_vocab_fr.txt', 'r', encoding='utf-8') as file:
    french_sentences = file.read().split('\n')

# Tokenization
tokenizer_eng = Tokenizer()
tokenizer_eng.fit_on_texts(english_sentences)
eng_sequences = tokenizer_eng.texts_to_sequences(english_sentences)

tokenizer_fr = Tokenizer()
tokenizer_fr.fit_on_texts(french_sentences)
fr_sequences = tokenizer_fr.texts_to_sequences(french_sentences)

# Padding sequences
max_len_eng = max(len(seq) for seq in eng_sequences)
max_len_fr = max(len(seq) for seq in fr_sequences)

eng_sequences_padded = pad_sequences(eng_sequences, maxlen=max_len_eng, padding='post')
fr_sequences_padded = pad_sequences(fr_sequences, maxlen=max_len_fr, padding='post')

# Vocabulary sizes
num_encoder_tokens = len(tokenizer_eng.word_index) + 1
num_decoder_tokens = len(tokenizer_fr.word_index) + 1

# Define model
latent_dim = 256

encoder_inputs = Input(shape=(None,))
encoder_emb = Embedding(num_encoder_tokens, latent_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_emb = Embedding(num_decoder_tokens, latent_dim, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_emb, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile and train the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit([eng_sequences_padded, fr_sequences_padded[:,:-1]], fr_sequences_padded[:,1:],
                    batch_size=64, epochs=1, validation_split=0.2)

# Train-test split
num_samples = len(eng_sequences_padded)
train_size = int(0.8 * num_samples)

eng_train, eng_test = eng_sequences_padded[:train_size], eng_sequences_padded[train_size:]
fr_train, fr_test = fr_sequences_padded[:train_size], fr_sequences_padded[train_size:]

# Evaluate the model on train and test sets
train_loss, train_accuracy = model.evaluate([eng_train, fr_train[:,:-1]], fr_train[:,1:])
test_loss, test_accuracy = model.evaluate([eng_test, fr_test[:,:-1]], fr_test[:,1:])

print("Train Loss:", train_loss)
print("Train Accuracy:", train_accuracy)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Calculate BLEU score on test set
#actual, predicted = [], []
#for i in range(len(eng_test)):
 #   input_seq = eng_test[i:i+1]
  #  actual_seq = fr_test[i:i+1, 1:]
   # decoded_sentence = decode_sequence(input_seq)
    #predicted.append(decoded_sentence.split())
    #actual.append([fr_test[i, j] for j in range(1, fr_test.shape[1]) if fr_test[i, j] != 0])

#bleu_score = corpus_bleu(actual, predicted)
#print("BLEU Score:", bleu_score)


862/862 [==============================] - 45s 52ms/step - loss: 0.1338 - accuracy: 0.9671
Train Loss: 0.12920048832893372
Train Accuracy: 0.9685085415840149
Test Loss: 0.13383717834949493
Test Accuracy: 0.9671372771263123


In [8]:
def translate_sentence(input_sentence):
    # Tokenize the input sentence
    input_sequence = tokenizer_eng.texts_to_sequences([input_sentence])
    # Pad the input sequence
    input_sequence = pad_sequences(input_sequence, maxlen=max_len_eng, padding='post')
    # Encode the input sequence
    states_value = encoder_model.predict(input_sequence)

    # Initialize the target sequence with start token
    target_sequence = np.zeros((1, 1))
    target_sequence[0, 0] = tokenizer_fr.word_index['<start>']

    stop_condition = False
    translated_sentence = ''

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_sequence] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = tokenizer_fr.index_word[sampled_token_index]

        if sampled_word != '<end>':
            translated_sentence += sampled_word + ' '

        # Exit condition: either hit max length or find stop token
        if sampled_word == '<end>' or len(translated_sentence.split()) > max_len_fr:
            stop_condition = True

        # Update the target sequence
        target_sequence = np.zeros((1, 1))
        target_sequence[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return translated_sentence.strip()

# Load the trained encoder and decoder models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_emb, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Prompt for user input
input_sentence = input("Enter an English sentence: ")

# Translate the input sentence
translated_sentence = translate_sentence(input_sentence)
print("Translated sentence:", translated_sentence)


Enter an English sentence: the
1/1 [==============================] - 2s 2s/step


KeyError: '<start>'